In [5]:
import os

app_token = os.environ.get('CHICAGO_API_TOKEN')

In [4]:
import json
import requests

def call_chicago_api(app_token):
    url = 'https://data.cityofchicago.org/resource/yhcw-iu53.json'
    params = {
        '$$app_token': app_token
    }

    try:
        response = requests.get(url, params=params, timeout=10)  # 10 seconds timeout
        if response.status_code == 200:
            # Successful request
            return response.json()
        else:
            # Error handling with more details
            return f"Error: {response.status_code}, Response: {response.text}"
    except requests.exceptions.RequestException as e:
        # Catching any request-related errors
        return f"Request Error: {e}"

# Replace 'your_app_token' with your actual app token
api_data = call_chicago_api(app_token)
pretty_data = json.dumps(api_data, indent=4)
print(pretty_data)

[
    {
        "id": "167913",
        "property_address": "10115 S Emerald Av",
        "submission_date": "2022-03-31T00:00:00.000",
        "owner_name": "US Bank National Association",
        "owner_date": "2022-03-24T00:00:00.000",
        "owner_management_agent_name": "Kathryn Florez",
        "owner_notices_agent_name": "Kathryn Florez",
        "owner_notices_agent_phone": "(312) 932-8408",
        "owner_notices_agent_email": "Kathryn.Florez@mccalla.com",
        "owner_address": "1 N Dearborn St",
        "owner_city": "Chicago",
        "owner_state": "IL",
        "owner_management_agent_address": "1 N Dearborn St",
        "owner_management_agent_city": "Chicago",
        "owner_management_agent_state": "IL",
        "location": "POINT (-87.64155776096058 41.71009504860621)"
    },
    {
        "id": "205832",
        "property_address": "10123 S Wentworth Av",
        "submission_date": "2022-04-18T00:00:00.000",
        "owner_name": "US Bank National Association",
 

In [27]:
import sqlite3
import requests
import json

def parse_point_data(point_str):
    if point_str.startswith("POINT (") and point_str.endswith(")"):
        point_str = point_str[7:-1]  # Remove 'POINT (' and the closing ')'
        longitude, latitude = map(float, point_str.split())
        return latitude, longitude
    return None, None

# Connect to SQLite Database
conn = sqlite3.connect('chicago_data.db')
c = conn.cursor()

# Insert or update data in the table
for item in api_data:
    latitude, longitude = parse_point_data(item.get('location', ''))
    c.execute('''INSERT OR REPLACE INTO foreclosures (id, property_address, submission_date, 
                 owner_name, owner_date, owner_management_agent_name, 
                 owner_notices_agent_name, owner_notices_agent_phone, 
                 owner_notices_agent_email, owner_address, 
                 owner_city, owner_state, owner_management_agent_address, 
                 owner_management_agent_city, owner_management_agent_state, 
                 latitude, longitude) 
                 VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', 
             (item.get('id'), item.get('property_address'), item.get('submission_date'), 
              item.get('owner_name'), item.get('owner_date'), item.get('owner_management_agent_name'), 
              item.get('owner_notices_agent_name'), item.get('owner_notices_agent_phone'), 
              item.get('owner_notices_agent_email'), item.get('owner_address'), 
              item.get('owner_city'), item.get('owner_state'), item.get('owner_management_agent_address'), 
              item.get('owner_management_agent_city'), item.get('owner_management_agent_state'), 
              latitude, longitude))

# Commit the changes and close the connection
conn.commit()
conn.close()


In [28]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('chicago_data.db')
c = conn.cursor()

# Execute a SELECT query
c.execute('SELECT * FROM foreclosures')

# Fetch all the rows returned by the query
rows = c.fetchall()

# Print each row
for row in rows:
    print(row)

# Close the connection
conn.close()

('167913', '10115 S Emerald Av', '2022-03-31T00:00:00.000', 'US Bank National Association', '2022-03-24T00:00:00.000', 'Kathryn Florez', 'Kathryn Florez', '(312) 932-8408', 'Kathryn.Florez@mccalla.com', '1 N Dearborn St', 'Chicago', 'IL', '1 N Dearborn St', 'Chicago', 'IL', None, 41.71009504860621, -87.64155776096058)
('205832', '10123 S Wentworth Av', '2022-04-18T00:00:00.000', 'US Bank National Association', '2022-04-11T00:00:00.000', 'Kathryn Florez', 'Kathryn Florez', '(312) 932-8408', 'Kathryn.florez@mccalla.com', '1 N Dearborn St', 'Chicago', 'IL', '1 N Dearborn St', 'Chicago', 'IL', None, 41.71004096296603, -87.6282427672632)
('479565', '10123 S Wentworth Av', '2022-09-20T00:00:00.000', 'U.S. Bank National Association', '2022-04-11T00:00:00.000', 'Victoria Ortiz', 'Victoria Ortiz', '(312) 476-5754', 'victoria.ortiz@mccalla.com', '1 N Dearborn St', 'Chicago', 'IL', '1 N Dearborn St', 'Chicago', 'IL', None, 41.71004096296603, -87.6282427672632)
('144236', '1015 E Hyde Park Bv', '2